In [1]:
# Imports
import numpy
import scipy.io
import math
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

print("Import Complete!")

Import Complete!


In [2]:
# Declarations

# Paths
SUBJECT_A_TRAIN_PATH = '../dataset/Matlab Format/Subject_A_Train.mat'
SUBJECT_A_TEST_PATH = '../dataset/Matlab Format/Subject_A_Test.mat'
SUBJECT_B_TRAIN_PATH = '../dataset/Matlab Format/Subject_B_Train.mat'
SUBJECT_B_TEST_PATH = '../dataset/Matlab Format/Subject_B_Test.mat'

# Input Constants
EPOCHS = 85
CHANNELS = 64
EXTRACTED_CHANNEL = 10 #Cz
EXTRACTED_CHANNELS = numpy.array([33, 10, 48, 50, 52, 55, 59, 61])
                    # numpy.array([33, 10, 48, 50, 52, 55, 59, 61])
                    # numpy.arange(64)
INTENSIFICATIONS = 12
START_WINDOW = 0
END_WINDOW = 192

SUBJECT_A_TEST_RESULTS = 'WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU'
SUBJECT_B_TEST_RESULTS = 'MERMIROOMUHJPXJOHUVLEORZP3GLOO7AUFDKEFTWEOOALZOP9ROCGZET1Y19EWX65QUYU7NAK_4YCJDVDNGQXODBEV2B5EFDIDNR'

# 6x6 Grid That Is Displayed To The User
MATRIX = numpy.array([['A', 'B', 'C', 'D', 'E', 'F'], \
                      ['G', 'H', 'I', 'J', 'K', 'L'], \
                      ['M', 'N', 'O', 'P', 'Q', 'R'], \
                      ['S', 'T', 'U', 'V', 'W', 'X'], \
                      ['Y', 'Z', '1', '2', '3', '4'], \
                      ['5', '6', '7', '8', '9', '_']])

print("Declaraion Complete!")

Declaraion Complete!


In [3]:
# Loading Data

Subject_A_Train = scipy.io.loadmat(SUBJECT_A_TRAIN_PATH)
Subject_A_Test = scipy.io.loadmat(SUBJECT_A_TEST_PATH)
Subject_B_Train = scipy.io.loadmat(SUBJECT_B_TRAIN_PATH)
Subject_B_Test = scipy.io.loadmat(SUBJECT_B_TEST_PATH)

print("Load Complete!")

Load Complete!


In [4]:
# Detection

# A
# Train
Subject_A_Signal_Train = numpy.array(Subject_A_Train.get('Signal'));
Subject_A_TargetChar_Train = numpy.array(Subject_A_Train.get('TargetChar'))[0];
Subject_A_StimulusCode_Train = numpy.array(Subject_A_Train.get('StimulusCode'));
print('Subject A Signal Train:', Subject_A_Signal_Train.shape)
print('Subject A TargetChar Train:', len(Subject_A_TargetChar_Train))
print('Subject A StimulusCode Train:', Subject_A_StimulusCode_Train.shape)

# Test
Subject_A_Signal_Test = numpy.array(Subject_A_Test.get('Signal'));
Subject_A_StimulusCode_Test = numpy.array(Subject_A_Test.get('StimulusCode'));
print('\nSubject A Signal Test:', Subject_A_Signal_Test.shape)
print('Subject A TargetChar Test:', len(SUBJECT_A_TEST_RESULTS))
print('Subject A StimulusCode Test:', Subject_A_StimulusCode_Test.shape)

# B
# Train
Subject_B_Signal_Train = numpy.array(Subject_B_Train.get('Signal'));
Subject_B_TargetChar_Train = numpy.array(Subject_B_Train.get('TargetChar'))[0];
Subject_B_StimulusCode_Train = numpy.array(Subject_B_Train.get('StimulusCode'));
print('\nSubject B Signal Train:', Subject_B_Signal_Train.shape)
print('Subject B TargetChar Train:', len(Subject_B_TargetChar_Train))
print('Subject B StimulusCode Train:', Subject_B_StimulusCode_Train.shape)

# Test
Subject_B_Signal_Test = numpy.array(Subject_B_Test.get('Signal'));
Subject_B_StimulusCode_Test = numpy.array(Subject_B_Test.get('StimulusCode'));
print('\nSubject B Signal Test:', Subject_B_Signal_Test.shape)
print('Subject B TargetChar Test:', len(SUBJECT_B_TEST_RESULTS))
print('Subject B StimulusCode Test:', Subject_B_StimulusCode_Test.shape)

print('\nDetect Complete!')

Subject A Signal Train: (85, 7794, 64)
Subject A TargetChar Train: 85
Subject A StimulusCode Train: (85, 7794)

Subject A Signal Test: (100, 7794, 64)
Subject A TargetChar Test: 100
Subject A StimulusCode Test: (100, 7794)

Subject B Signal Train: (85, 7794, 64)
Subject B TargetChar Train: 85
Subject B StimulusCode Train: (85, 7794)

Subject B Signal Test: (100, 7794, 64)
Subject B TargetChar Test: 100
Subject B StimulusCode Test: (100, 7794)

Detect Complete!


In [9]:
# Averaging

# Function To Get Average Of All Signals (And Channels) Within 1 Run
def calculate_runs_average(
        signals,
        stimulus_code,
        matrix,
        moving_average_filter = 5,
        decimation_filter = 4,
        repeatitions = 15,
        digitization_samples = 240,
        start_window = 0,
        end_window = 240
    ):
    
    window = end_window - start_window
    intensifications = matrix.shape[0] + matrix.shape[1]
    responses = numpy.zeros((signals.shape[0], intensifications, window, signals.shape[2]))
    decimated_responses = numpy.zeros((signals.shape[0], intensifications, math.floor(window / decimation_filter), signals.shape[2]))
    digitization_difference = math.floor(digitization_samples / 10)
    
    for epoch in range(signals.shape[0]):
        
        # Sum of All Repeatitions -> Division of Sum -> Average
        intensification_counter = numpy.zeros((intensifications))
        for n in range(1, signals.shape[1]):
            if stimulus_code[epoch, n] == 0 and stimulus_code[epoch, n - 1] != 0:
                intensification_counter[int(stimulus_code[epoch, n - 1]) - 1] += 1
                responses[epoch, int(stimulus_code[epoch, n - 1]) - 1] += signals[epoch, n + start_window - digitization_difference : n + end_window - digitization_difference]
        for intensification in range(intensifications):
            responses[epoch, intensification] /= intensification_counter[intensification]
        
        # Calculation of Common Average Reference
        mean_sample = numpy.mean(responses[epoch, :, :, :], axis=2)
        for channel in range(responses.shape[3]):
            responses[epoch, :, :, channel] = responses[epoch, :, :, channel] - mean_sample
        
        # Calculation of Moving Average
        for intensification in range(responses.shape[1]):
            for sample in range(math.ceil(responses.shape[2] / moving_average_filter)):
                if ((sample * moving_average_filter) + moving_average_filter) < responses.shape[2]:
                    responses[epoch, intensification, (sample * moving_average_filter) : ((sample * moving_average_filter) + moving_average_filter), :] = \
                        numpy.mean(responses[epoch, intensification, (sample * moving_average_filter) : ((sample * moving_average_filter) + moving_average_filter), :], axis=0)
                else:
                    responses[epoch, intensification, (sample * moving_average_filter) : , :] = \
                        numpy.mean(responses[epoch, intensification, (sample * moving_average_filter) : , :], axis=0)
        
        # Calculation of Z-Score
        mean_channel = numpy.mean(responses[epoch, :, :, :], axis=1)
        std_channel = numpy.std(responses[epoch, :, :, :], axis=1)
        for sample in range(responses.shape[2]):
            responses[epoch, :, sample, :] = (responses[epoch, :, sample, :] - mean_channel) / std_channel
        
        # Application of Decimation
        for intensification in range(responses.shape[1]):
            for i in range(math.floor(window / decimation_filter)):
                decimated_responses[epoch, intensification, i, :] = responses[epoch, intensification, (i * decimation_filter), :]
    
    return decimated_responses

# A
# Train
Subject_A_Average_Signal_Train = calculate_runs_average(
    Subject_A_Signal_Train,
    Subject_A_StimulusCode_Train,
    MATRIX,
    start_window=START_WINDOW,
    end_window=END_WINDOW
)
print('Subject A Average Signal Train:', Subject_A_Average_Signal_Train.shape)
# Test
Subject_A_Average_Signal_Test = calculate_runs_average(
    Subject_A_Signal_Test,
    Subject_A_StimulusCode_Test,
    MATRIX,
    start_window=START_WINDOW,
    end_window=END_WINDOW
)
print('Subject A Average Signal Test:', Subject_A_Average_Signal_Test.shape)

# B
# Train
Subject_B_Average_Signal_Train = calculate_runs_average(
    Subject_B_Signal_Train,
    Subject_B_StimulusCode_Train,
    MATRIX,
    start_window=START_WINDOW,
    end_window=END_WINDOW
)
print('\nSubject B Average Signal Train:', Subject_B_Average_Signal_Train.shape)
# Test
Subject_B_Average_Signal_Test = calculate_runs_average(
    Subject_B_Signal_Test,
    Subject_B_StimulusCode_Test,
    MATRIX,
    start_window=START_WINDOW,
    end_window=END_WINDOW
)
print('Subject B Average Signal Test:', Subject_B_Average_Signal_Test.shape)

print('\nAverage Complete!')

Subject A Average Signal Train: (85, 12, 48, 64)
Subject A Average Signal Test: (100, 12, 48, 64)

Subject B Average Signal Train: (85, 12, 48, 64)
Subject B Average Signal Test: (100, 12, 48, 64)

Average Complete!


In [10]:
# Function To Flatten The  Channels
def flatten_channels(
        average_signals,
        extracted_channels
    ):
    
    flattened_average_signals = numpy.zeros((average_signals.shape[0], average_signals.shape[1], average_signals.shape[2] * extracted_channels.shape[0]))
    
    for epoch in range(average_signals.shape[0]):
        
        for intensification in range(average_signals.shape[1]):
            
            for channel_index in range(extracted_channels.shape[0]):
                flattened_average_signals[epoch, intensification, (average_signals.shape[2] * channel_index):(average_signals.shape[2] * channel_index) + average_signals.shape[2]] = \
                    average_signals[epoch, intensification, :, extracted_channels[channel_index]]
    
    return flattened_average_signals

# A
# Train
Subject_A_Flatten_Signal_Train = flatten_channels(
    Subject_A_Average_Signal_Train,
    EXTRACTED_CHANNELS
)
print('Subject A Flatten Signal Train:', Subject_A_Flatten_Signal_Train.shape)
# Test
Subject_A_Flatten_Signal_Test = flatten_channels(
    Subject_A_Average_Signal_Test,
    EXTRACTED_CHANNELS
)
print('Subject A Flatten Signal Test:', Subject_A_Flatten_Signal_Test.shape)

# B
# Train
Subject_B_Flatten_Signal_Train = flatten_channels(
    Subject_B_Average_Signal_Train,
    EXTRACTED_CHANNELS
)
print('\nSubject B Flatten Signal Train:', Subject_B_Flatten_Signal_Train.shape)
# Test
Subject_B_Flatten_Signal_Test = flatten_channels(
    Subject_B_Average_Signal_Test,
    EXTRACTED_CHANNELS
)
print('Subject B Flatten Signal Test:', Subject_B_Flatten_Signal_Test.shape)

print('\nFlatten Complete!')

Subject A Flatten Signal Train: (85, 12, 384)
Subject A Flatten Signal Test: (100, 12, 384)

Subject B Flatten Signal Train: (85, 12, 384)
Subject B Flatten Signal Test: (100, 12, 384)

Flatten Complete!


In [7]:
# A
# Train
Subject_A_Reshape_Signal_Train = numpy.reshape(
    Subject_A_Flatten_Signal_Train,
    (Subject_A_Flatten_Signal_Train.shape[0] * Subject_A_Flatten_Signal_Train.shape[1], -1)
)
print('Subject A Reshape Signal Train:', Subject_A_Reshape_Signal_Train.shape)
# Test
Subject_A_Reshape_Signal_Test = numpy.reshape(
    Subject_A_Flatten_Signal_Test,
    (Subject_A_Flatten_Signal_Test.shape[0] * Subject_A_Flatten_Signal_Test.shape[1], -1)
)
print('Subject A Reshape Signal Test:', Subject_A_Reshape_Signal_Test.shape)

# B
# Train
Subject_B_Reshape_Signal_Train = numpy.reshape(
    Subject_B_Flatten_Signal_Train,
    (Subject_B_Flatten_Signal_Train.shape[0] * Subject_B_Flatten_Signal_Train.shape[1], -1)
)
print('\nSubject B Reshape Signal Train:', Subject_B_Reshape_Signal_Train.shape)
# Test
Subject_B_Reshape_Signal_Test = numpy.reshape(
    Subject_B_Flatten_Signal_Test,
    (Subject_B_Flatten_Signal_Test.shape[0] * Subject_B_Flatten_Signal_Test.shape[1], -1)
)
print('Subject b Reshape Signal Test:', Subject_B_Reshape_Signal_Test.shape)

print('\nReshape Signal Complete!')

Subject A Reshape Signal Train: (1020, 384)
Subject A Reshape Signal Test: (1200, 384)

Subject B Reshape Signal Train: (1020, 384)
Subject b Reshape Signal Test: (1200, 384)

Reshape Signal Complete!


In [8]:
def construct_classes(
        average_signals,
        target_char,
        matrix
    ):
    
    intensifications = matrix.shape[0] + matrix.shape[1]
    
    classes = numpy.zeros((average_signals.shape[0], intensifications))
    
    for epoch in range(average_signals.shape[0]):
        
        # Getting Index Of Chosen Character
        indices = numpy.where(matrix == target_char[epoch])
        chosen_column = indices[1][0]
        chosen_row = indices[0][0] + matrix.shape[1]
        
        for row_column in range(intensifications):
            if row_column == chosen_row or row_column == chosen_column:
                classes[epoch, row_column] = 1
            else:
                classes[epoch, row_column] = -1
    
    return classes

# A
# Train
Subject_A_Classes_Train = construct_classes(
    Subject_A_Average_Signal_Train,
    Subject_A_TargetChar_Train,
    MATRIX
)
print('Subject A Classes Train:', Subject_A_Classes_Train.shape)
# Test
Subject_A_Classes_Test = construct_classes(
    Subject_A_Average_Signal_Test,
    SUBJECT_A_TEST_RESULTS,
    MATRIX
)
print('Subject A Classes Test:', Subject_A_Classes_Test.shape)

# B
# Train
Subject_B_Classes_Train = construct_classes(
    Subject_B_Average_Signal_Train,
    Subject_B_TargetChar_Train,
    MATRIX
)
print('\nSubject B Classes Train:', Subject_B_Classes_Train.shape)
# Test
Subject_B_Classes_Test = construct_classes(
    Subject_B_Average_Signal_Test,
    SUBJECT_B_TEST_RESULTS,
    MATRIX
)
print('Subject B Classes Test:', Subject_B_Classes_Test.shape)

print('\nClasses Construct Complete!')

Subject A Classes Train: (85, 12)
Subject A Classes Test: (100, 12)

Subject B Classes Train: (85, 12)
Subject B Classes Test: (100, 12)

Classes Construct Complete!


In [9]:
# A
# Train
Subject_A_Reshape_Classes_Train = numpy.reshape(
    Subject_A_Classes_Train,
    (Subject_A_Classes_Train.shape[0] * Subject_A_Classes_Train.shape[1])
)
print('Subject A Reshape Classes Train:', Subject_A_Reshape_Classes_Train.shape)
# Test
Subject_A_Reshape_Classes_Test = numpy.reshape(
    Subject_A_Classes_Test,
    (Subject_A_Classes_Test.shape[0] * Subject_A_Classes_Test.shape[1])
)
print('Subject A Reshape Classes Test:', Subject_A_Reshape_Classes_Test.shape)

# B
# Train
Subject_B_Reshape_Classes_Train = numpy.reshape(
    Subject_B_Classes_Train,
    (Subject_B_Classes_Train.shape[0] * Subject_B_Classes_Train.shape[1])
)
print('\nSubject B Reshape Classes Train:', Subject_B_Reshape_Classes_Train.shape)
# Test
Subject_B_Reshape_Classes_Test = numpy.reshape(
    Subject_B_Classes_Test,
    (Subject_B_Classes_Test.shape[0] * Subject_B_Classes_Test.shape[1])
)
print('Subject B Reshape Classes Test:', Subject_B_Reshape_Classes_Test.shape)

print('\nReshape Classes Complete!')

Subject A Reshape Classes Train: (1020,)
Subject A Reshape Classes Test: (1200,)

Subject B Reshape Classes Train: (1020,)
Subject B Reshape Classes Test: (1200,)

Reshape Classes Complete!


In [10]:
# A
Subject_A_Model = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
Subject_A_Model.fit(Subject_A_Reshape_Signal_Train, Subject_A_Reshape_Classes_Train)

# B
Subject_B_Model = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
Subject_B_Model.fit(Subject_B_Reshape_Signal_Train, Subject_B_Reshape_Classes_Train)

print('Model Fit Complete!')

Model Fit Complete!


In [11]:
# Function To Predict Character
def predict_characters(
        flatten_signal,
        model,
        matrix
    ):
    
    out = ''
    for epoch in range(flatten_signal.shape[0]):

        predicted_classes = model.decision_function(flatten_signal[epoch])

        column = numpy.where(predicted_classes[:6] == numpy.amax(predicted_classes[:6]))[0][0]
        row = numpy.where(predicted_classes[6:] == numpy.amax(predicted_classes[6:]))[0][0]
        
        out += matrix[row, column]

    return out

# A
# Train
Subject_A_Predict_Character_Train = predict_characters(
    Subject_A_Flatten_Signal_Train,
    Subject_A_Model,
    MATRIX
)
print('Subject A Target Characters Train :', Subject_A_TargetChar_Train)
print('Subject A Predict Characters Train:', Subject_A_Predict_Character_Train)
# Test
Subject_A_Predict_Character_Test = predict_characters(
    Subject_A_Flatten_Signal_Test,
    Subject_A_Model,
    MATRIX
)
print('\nSubject A Target Characters Test :', SUBJECT_A_TEST_RESULTS)
print('Subject A Predict Characters Test:', Subject_A_Predict_Character_Test)

# B
# Train
Subject_B_Predict_Character_Train = predict_characters(
    Subject_B_Flatten_Signal_Train,
    Subject_B_Model,
    MATRIX
)
print('\nSubject B Target Characters Train :', Subject_B_TargetChar_Train)
print('Subject B Predict Characters Train:', Subject_B_Predict_Character_Train)
# Test
Subject_B_Predict_Character_Test = predict_characters(
    Subject_B_Flatten_Signal_Test,
    Subject_B_Model,
    MATRIX
)
print('\nSubject B Target Characters Test :', SUBJECT_B_TEST_RESULTS)
print('Subject B Predict Characters Test:', Subject_B_Predict_Character_Test)

print('\nCharacter Predict Complete!')

Subject A Target Characters Train : EAEVQTDOJG8RBRGONCEDHCTUIDBPUHMEM6OUXOCFOUKWA4VJEFRZROLHYNQDW_EKTLBWXEPOUIKZERYOOTHQI
Subject A Predict Characters Train: EAEVQTDOJG8RBRGONCEDHCTUIDBPUHMEM6OUXOCFOUKXA4VJEFRZROLHYNQDW_WKTLBWXEPOUIKZERYOOTHQI

Subject A Target Characters Test : WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU
Subject A Predict Characters Test: WQXPLZCOMRKOR7YCZDEZ1DPC9NNVGRJDHDUV4MEUO1OJJ2OFYPOO6J7LDGYEAOA5VZNEVBTXOO1TDOOLUEE6ZFA33XAW_K1R3MRU

Subject B Target Characters Train : VGREAAH8TVRHBYN_UGCOLO4EUERDOOHCIFOMDNU6LQCPKEIREKOYRQIDJXPBKOJDWZEUEWWFOEBHXTQTTZUMO
Subject B Predict Characters Train: VGREEAH8TVRHBYN_UGCOLO4EUERDOOHCIFOMDNU6LQCPKEIREKOYRQIDJXPBKOJDWUEUEWWFOEBHXTQTTZUMO

Subject B Target Characters Test : MERMIROOMUHJPXJOHUVLEORZP3GLOO7AUFDKEFTWEOOALZOP9ROCGZET1Y19EWX65QUYU7NAK_4YCJDVDNGQXODBEV2B5EFDIDNR
Subject B Predict Characters Test: GERMIROOMUHJPXJOHUPL1ORZP3GLOO5AUFDKEFNWEOOALZOP9ROCGZEQ1Y19EWX65Q

In [12]:
# Function To Calculate Accuracy
def calculate_accuracy(
        target_characters,
        predicted_characters
    ):
    
    accuracy = 0
    for i in range(len(target_characters)):
        if target_characters[i] == predicted_characters[i]:
            accuracy += 1
    accuracy = (accuracy / len(target_characters)) * 100
    
    return accuracy

# A
# Train
print('Subject A Prediction Accuracy Train:', calculate_accuracy(Subject_A_TargetChar_Train, Subject_A_Predict_Character_Train))
# Test
print('Subject A Prediction Accuracy Test:', calculate_accuracy(SUBJECT_A_TEST_RESULTS, Subject_A_Predict_Character_Test))

# B
# Train
print('Subject B Prediction Accuracy Train:', calculate_accuracy(Subject_B_TargetChar_Train, Subject_B_Predict_Character_Train))
# Test
print('Subject B Prediction Accuracy Test:', calculate_accuracy(SUBJECT_B_TEST_RESULTS, Subject_B_Predict_Character_Test))

print('\nAccuracy Calculation Complete!')

Subject A Prediction Accuracy Train: 97.6470588235294
Subject A Prediction Accuracy Test: 81.0
Subject B Prediction Accuracy Train: 97.6470588235294
Subject B Prediction Accuracy Test: 91.0

Accuracy Calculation Complete!
